In [ ]:
import csv
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = 103501

In [ ]:
outage_csv = r"C:\EsriTraining\PYTS\North_Carolina\OutageXY.csv"
outage_coords = []
csv_file = open(outage_csv)
csv_reader = csv.reader(csv_file)
next(csv_reader)
for row in csv_reader:
    outage_coords.append(row)

In [ ]:
pnt_list = []
for coord in outage_coords:
    pnt = arcpy.Point(coord[0], coord[1])
    pnt_list.append(pnt)
pnt_ary = arcpy.Array(pnt_list)
outage_multipoint = arcpy.Multipoint(pnt_ary)

In [ ]:
outage_area = "CurrentAffectedAreas"
convex_hull = outage_multipoint.convexHull()
arcpy.CopyFeatures_management(convex_hull, outage_area)

In [ ]:
contact_fc = "AffectedParcels"
gdb = r"C:\EsriTraining\PYTS\North_Carolina\CountyData.gdb"
arcpy.CreateFeatureclass_management(gdb, contact_fc, "POINT")
arcpy.AddFields_management(f"{gdb}\\{contact_fc}",[["Owner_Name", "TEXT"], ["Phone_Number", "TEXT"]])

In [ ]:
parcels = "ParcelPts"
with arcpy.da.SearchCursor(parcels, ["SHAPE@XY", "Owner_Name", "Phone_Number"]) as parcel_cursor:
    for parcel in parcel_cursor:
        pts = arcpy.Point((parcel[0])[0],(parcel[0])[1])
        parcel_geoms = arcpy.PointGeometry(pts)
        if parcel_geoms.within(convex_hull):
            cursor = arcpy.da.InsertCursor(f"{gdb}\\{contact_fc}", ["SHAPE@XY", "Owner_Name", "Phone_Number"])
            cursor.insertRow([((parcel[0])[0], (parcel[0])[1]), parcel[1], parcel[2]])
            del cursor
        else:
            continue
del parcel_cursor